# Example for:
Transfer Learning Empirical Experiment from all other classes of FashionMNIST to shoes (Sandal, Sneaker, Ankle boot) classes in FashionMNIST

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64, # num channels for CNN
      'two_linear_layers': False,
      'hidden_dim_lin': 128,  # if two_linear_layers == True
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': [0, 1, 2, 3, 4, 6, 8],
      'fine_tune_classes': [5, 7, 9],
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      # default value for pooling_every_n_layers is 1 (after each cnn layer)
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': False,         # CHANGE: reinit the conv lyers only after the cut
      'reinit_both_dense': True   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.FashionMNIST, datasets.FashionMNIST, root_dir, transform=transform)## Pretraining

In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

37800

## Pretraining

In [6]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=7, bias=True)
)

In [7]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 35.97%
Validation Accuracy: 36.63%
Epoch: 1 	Training Accuracy: 59.17%
Validation Accuracy: 59.93%
Epoch: 2 	Training Accuracy: 69.57%
Validation Accuracy: 69.97%
Epoch: 3 	Training Accuracy: 76.40%
Validation Accuracy: 76.27%
Epoch: 4 	Training Accuracy: 79.24%
Validation Accuracy: 79.20%
Epoch: 5 	Training Accuracy: 79.97%
Validation Accuracy: 79.83%
Epoch: 6 	Training Accuracy: 78.35%
Validation Accuracy: 78.00%
Epoch: 7 	Training Accuracy: 82.17%
Validation Accuracy: 82.20%
Epoch: 8 	Training Accuracy: 83.48%
Validation Accuracy: 83.03%
Epoch: 9 	Training Accuracy: 83.65%
Validation Accuracy: 82.93%
Epoch: 10 	Training Accuracy: 84.15%
Validation Accuracy: 83.87%
Epoch: 11 	Training Accuracy: 84.84%
Validation Accuracy: 84.03%
Epoch: 12 	Training Accuracy: 85.16%
Validation Accuracy: 84.83%
Epoch: 13 	Training Accuracy: 85.78%
Validation Accuracy: 85.33%
Epoch: 14 	Training Accuracy: 86.27%
Validation Accuracy: 85.90%
Epoch: 15 	Training Accuracy: 86.53

In [8]:
torch.save(pretrained_model.state_dict(), 'pretrained_models/pretrained_cifar_arc/pretrained_model_toshoes.pth')

In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_cifar_arc/pretrained_model_toshoes.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=7, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.3543, Accuracy: 3679.0/4200 (88%)

              precision    recall  f1-score   support

     Class 0       0.84      0.86      0.85       560
     Class 1       0.99      0.97      0.98       638
     Class 2       0.85      0.86      0.86       595
     Class 3       0.86      0.94      0.90       615
     Class 4       0.82      0.83      0.82       579
     Class 5       0.80      0.69      0.74       626
     Class 6       0.98      0.98      0.98       587

    accuracy                           0.88      4200
   macro avg       0.87      0.88      0.87      4200
weighted avg       0.88      0.88      0.87      4200



0.8759523809523809

In [16]:
# save model for later use
foldername = "pretrained_models/pretrained_normal_classes_new"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [9]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]


In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0


KeyboardInterrupt: 

In [ ]:
print(baselines_results)

In [21]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results_jsons/baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [9]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [1]


In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9598, Test Accuracy: 0.9593

Sampled Percentage: 1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Early stopping invoked.
Training Accuracy: 0.9597, Test Accuracy: 0.9598

Sampled Percentage: 1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Early stopping invoked.
Training Accuracy: 0.9620, Test Accuracy: 0.9623

Sampled Percentage: 1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Training Accuracy: 0.9560, Test Accuracy: 0.9558

Sampled Percentage: 1, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Training Accuracy: 0.9575, Test Accuracy: 0.9578

Sampled Percentage: 1, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9612, Test Accuracy: 0.9606

Sampled Percentage: 1, Sampled Cut Point: 1, Lr: 0.001, Repeat: 1
Early stopping invoked.
Training Accuracy: 0.9576, Test Accuracy: 0.9576

Sampled Percentage: 1, Sampled Cut Point: 1, Lr: 0.001, Repeat: 2
Early stopping invoked.
Training Accuracy: 0.9591

In [11]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9598048900963201, 'test_acc': 0.9593333333333334}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9597431464559151, 'test_acc': 0.9598333333333333}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9620276611509014, 'test_acc': 0.9622777777777778}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9560385280316127, 'test_acc': 0.9558333333333333}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9575203754013336, 'test_acc': 0.9577777777777777}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.961163250185231, 'test_acc': 0.9605555555555556}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9575821190417387, 'test_acc': 0.9575555555555556}, {'lr':

In [12]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9598048900963201, 'test_acc': 0.9593333333333334}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9597431464559151, 'test_acc': 0.9598333333333333}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9620276611509014, 'test_acc': 0.9622777777777778}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9560385280316127, 'test_acc': 0.9558333333333333}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9575203754013336, 'test_acc': 0.9577777777777777}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.961163250185231, 'test_acc': 0.9605555555555556}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9575821190417387, 'test_acc': 0.9575555555555556}, {'lr':

In [14]:
baseline_results = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.7333333333333333, 'test_acc': 0.7913333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.8666666666666667, 'test_acc': 0.7685555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.8, 'test_acc': 0.7598333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.6666666666666666, 'test_acc': 0.7891111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8666666666666667, 'test_acc': 0.7294444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8191111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.8, 'test_acc': 0.7269444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.8, 'test_acc': 0.7956111111111112}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.7333333333333333, 'test_acc': 0.7578888888888888}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.8, 'test_acc': 0.7578333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.8666666666666667, 'test_acc': 0.7905555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.8666666666666667, 'test_acc': 0.7702222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.8666666666666667, 'test_acc': 0.7936111111111112}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.7825555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.6666666666666666, 'test_acc': 0.7353333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.8, 'test_acc': 0.7211666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.7427777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.9333333333333333, 'test_acc': 0.7966111111111112}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.6666666666666666, 'test_acc': 0.7211111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.8666666666666667, 'test_acc': 0.813}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.9333333333333333, 'test_acc': 0.7835555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.9333333333333333, 'test_acc': 0.7707222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.9333333333333333, 'test_acc': 0.7913888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.8, 'test_acc': 0.7904444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.9333333333333333, 'test_acc': 0.7719444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.7839506172839507, 'test_acc': 0.7858333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.7901234567901234, 'test_acc': 0.7962777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.808641975308642, 'test_acc': 0.7831666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.8024691358024691, 'test_acc': 0.7898333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.7592592592592593, 'test_acc': 0.7720555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.845679012345679, 'test_acc': 0.8468888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.7839506172839507, 'test_acc': 0.7595}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.8518518518518519, 'test_acc': 0.8443333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.8271604938271605, 'test_acc': 0.8089444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.8333333333333334, 'test_acc': 0.778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.8580246913580247, 'test_acc': 0.8251666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.7901234567901234, 'test_acc': 0.7702222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.7716049382716049, 'test_acc': 0.782}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.8271604938271605, 'test_acc': 0.7667222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.7654320987654321, 'test_acc': 0.7632222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.8271604938271605, 'test_acc': 0.8294444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.8703703703703703, 'test_acc': 0.8538333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.7962962962962963, 'test_acc': 0.7880555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.7592592592592593, 'test_acc': 0.8167222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.8395061728395061, 'test_acc': 0.8371666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.808641975308642, 'test_acc': 0.8075}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.8703703703703703, 'test_acc': 0.8307777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.8580246913580247, 'test_acc': 0.8182222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.845679012345679, 'test_acc': 0.8308333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.9074074074074074, 'test_acc': 0.8415}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8487654320987654, 'test_acc': 0.8469444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.8530864197530864, 'test_acc': 0.8434444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.7722222222222223, 'test_acc': 0.7762777777777777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.832716049382716, 'test_acc': 0.8271666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.7487654320987654, 'test_acc': 0.7597222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.8438271604938271, 'test_acc': 0.8552777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.7574074074074074, 'test_acc': 0.7420555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.845679012345679, 'test_acc': 0.8425}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.7450617283950617, 'test_acc': 0.7429444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.8574074074074074, 'test_acc': 0.851}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.8567901234567902, 'test_acc': 0.8386666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.7530864197530864, 'test_acc': 0.7797222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.7722222222222223, 'test_acc': 0.7838888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.7858024691358024, 'test_acc': 0.7668888888888888}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.7617283950617284, 'test_acc': 0.7717777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.8376543209876544, 'test_acc': 0.8371666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.8728395061728395, 'test_acc': 0.8602777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.782716049382716, 'test_acc': 0.7853888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.7654320987654321, 'test_acc': 0.7621111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.8382716049382716, 'test_acc': 0.8364444444444444}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8581481481481481, 'test_acc': 0.8575555555555555}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.8955555555555555, 'test_acc': 0.8921666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.7662962962962963, 'test_acc': 0.7716111111111111}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.8920987654320988, 'test_acc': 0.8893333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8783950617283951, 'test_acc': 0.8782222222222222}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9428253889849345, 'test_acc': 0.9428888888888889}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.956408989874043, 'test_acc': 0.9566111111111111}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9478883674981476, 'test_acc': 0.9482777777777778}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9472709310940973, 'test_acc': 0.9474444444444444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9418374907384539, 'test_acc': 0.9417777777777778}]
finetune1 = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9333333333333333, 'test_acc': 0.7666666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8666666666666667, 'test_acc': 0.769}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.7745555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8666666666666667, 'test_acc': 0.8476666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.8462222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8326666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.6, 'test_acc': 0.6354444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.8666666666666667, 'test_acc': 0.774}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.8090555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.8, 'test_acc': 0.8178333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.6666666666666666, 'test_acc': 0.6388333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.7333333333333333, 'test_acc': 0.8451111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.769}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.8669444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.6, 'test_acc': 0.6286111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.8666666666666667, 'test_acc': 0.8152777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.8222777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.8666666666666667, 'test_acc': 0.8128333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.6873333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.7523888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.9333333333333333, 'test_acc': 0.8281666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.8313888888888888}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.8666666666666667, 'test_acc': 0.7552777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.9333333333333333, 'test_acc': 0.8651666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8492222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8666666666666667, 'test_acc': 0.775}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.8121111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.8344444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.8, 'test_acc': 0.8453333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9333333333333333, 'test_acc': 0.8514444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8297777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8189444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.7333333333333333, 'test_acc': 0.7215}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.8141666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.8, 'test_acc': 0.8378333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.864}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.6, 'test_acc': 0.6577777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.7948888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.8696666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.6666666666666666, 'test_acc': 0.7098333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.8385555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.8257777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9333333333333333, 'test_acc': 0.8105}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.7333333333333333, 'test_acc': 0.7213888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.6666666666666666, 'test_acc': 0.6753888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.9333333333333333, 'test_acc': 0.8392777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.9333333333333333, 'test_acc': 0.819}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.9333333333333333, 'test_acc': 0.7683888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.8715}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8479444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8666666666666667, 'test_acc': 0.8062777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8666666666666667, 'test_acc': 0.8114444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9333333333333333, 'test_acc': 0.8222222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9333333333333333, 'test_acc': 0.8602777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9333333333333333, 'test_acc': 0.8515555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8305}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8315555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.7333333333333333, 'test_acc': 0.7427222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.8080555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.6, 'test_acc': 0.7613888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.8773333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.8, 'test_acc': 0.8506666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.8525}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.9333333333333333, 'test_acc': 0.8705555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.6, 'test_acc': 0.7144444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9333333333333333, 'test_acc': 0.8302777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.8503888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.9333333333333333, 'test_acc': 0.8350555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.8, 'test_acc': 0.8372222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.7333333333333333, 'test_acc': 0.8126111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.9333333333333333, 'test_acc': 0.8537777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.8543333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.9333333333333333, 'test_acc': 0.7751111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.8, 'test_acc': 0.8716666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.9333333333333333, 'test_acc': 0.8281111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8, 'test_acc': 0.7682222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8666666666666667, 'test_acc': 0.8323888888888888}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.8262777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8515555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.8570555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8344444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8413888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.8021111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.8224444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.9333333333333333, 'test_acc': 0.8630555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.9333333333333333, 'test_acc': 0.8652777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.8, 'test_acc': 0.857}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.8634444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.9333333333333333, 'test_acc': 0.8751111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.8, 'test_acc': 0.7849444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.8195}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.8297222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.8055}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.8728333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.7333333333333333, 'test_acc': 0.8517777777777777}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.8666666666666667, 'test_acc': 0.8407222222222223}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.8416111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.8666666666666667, 'test_acc': 0.81}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.8666666666666667, 'test_acc': 0.8720555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8030555555555555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9333333333333333, 'test_acc': 0.8137777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.8112777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.7877777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.8496666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9333333333333333, 'test_acc': 0.8598333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.8, 'test_acc': 0.7414444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8167777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.8332777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8666666666666667, 'test_acc': 0.7976666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.9333333333333333, 'test_acc': 0.8608333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9333333333333333, 'test_acc': 0.8761666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8666666666666667, 'test_acc': 0.8452777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.8556111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9333333333333333, 'test_acc': 0.8663888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.7506111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.8666666666666667, 'test_acc': 0.7957777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.8041666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.8223333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.8, 'test_acc': 0.8058333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8666666666666667, 'test_acc': 0.8428333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.8666666666666667, 'test_acc': 0.8369444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.7823333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.8666666666666667, 'test_acc': 0.8222222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.9333333333333333, 'test_acc': 0.8703888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.8142222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8, 'test_acc': 0.804}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9333333333333333, 'test_acc': 0.8022777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.8161666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9333333333333333, 'test_acc': 0.844}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9333333333333333, 'test_acc': 0.8502222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.7333333333333333, 'test_acc': 0.7058333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.8148333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.8186111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.9333333333333333, 'test_acc': 0.8063333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9333333333333333, 'test_acc': 0.8691666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.8804444444444445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.8666666666666667, 'test_acc': 0.8433888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.844}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.848}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.7333333333333333, 'test_acc': 0.7416111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.8502222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.8283333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.8344444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.8657777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.8666666666666667, 'test_acc': 0.8633333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.8, 'test_acc': 0.7392222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.8115555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.7333333333333333, 'test_acc': 0.758}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.8666666666666667, 'test_acc': 0.8746666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.8666666666666667, 'test_acc': 0.8078333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9333333333333333, 'test_acc': 0.793}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8666666666666667, 'test_acc': 0.82}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.8012222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8666666666666667, 'test_acc': 0.8393333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9333333333333333, 'test_acc': 0.8013333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.8154444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.9333333333333333, 'test_acc': 0.7756666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.9333333333333333, 'test_acc': 0.8331666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.8666666666666667, 'test_acc': 0.8141111111111111}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.8627222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.9333333333333333, 'test_acc': 0.8595}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.8666666666666667, 'test_acc': 0.8137222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.9333333333333333, 'test_acc': 0.8133333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.8448888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.8, 'test_acc': 0.79}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.9333333333333333, 'test_acc': 0.8424444444444444}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.828}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.8102777777777778}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.8583333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8, 'test_acc': 0.8123888888888889}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.8, 'test_acc': 0.7552222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.7500555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.9333333333333333, 'test_acc': 0.7885555555555556}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.8666666666666667, 'test_acc': 0.8542222222222222}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.7923888888888889}]
finetune2 = [{'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9259259259259259, 'test_acc': 0.8687222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.8827160493827161, 'test_acc': 0.8653333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8703703703703703, 'test_acc': 0.8743333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9012345679012346, 'test_acc': 0.8726111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8765432098765432, 'test_acc': 0.872}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.7901234567901234, 'test_acc': 0.7989444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.9753086419753086, 'test_acc': 0.8843888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.9012345679012346, 'test_acc': 0.8683888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.8888888888888888, 'test_acc': 0.8433888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.8703703703703703, 'test_acc': 0.8528333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.9382716049382716, 'test_acc': 0.8852777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.9197530864197531, 'test_acc': 0.897}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.8641975308641975, 'test_acc': 0.8505555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.6604938271604939, 'test_acc': 0.6766111111111112}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.845679012345679, 'test_acc': 0.8337777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.9197530864197531, 'test_acc': 0.8673888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.8703703703703703, 'test_acc': 0.8163333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.8950617283950617, 'test_acc': 0.8487222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.9197530864197531, 'test_acc': 0.886}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.8333333333333334, 'test_acc': 0.8477222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.8888888888888888, 'test_acc': 0.868}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.6851851851851852, 'test_acc': 0.7023333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.8209876543209876, 'test_acc': 0.7923333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.8395061728395061, 'test_acc': 0.8492777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.9135802469135802, 'test_acc': 0.8712777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9259259259259259, 'test_acc': 0.87}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.8395061728395061, 'test_acc': 0.8471666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.8703703703703703, 'test_acc': 0.8728333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9012345679012346, 'test_acc': 0.8721666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.8888888888888888, 'test_acc': 0.8720555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.9135802469135802, 'test_acc': 0.8550555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9567901234567902, 'test_acc': 0.8824444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.9567901234567902, 'test_acc': 0.8841666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.9012345679012346, 'test_acc': 0.8438333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.9259259259259259, 'test_acc': 0.8709444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9135802469135802, 'test_acc': 0.8943333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.9444444444444444, 'test_acc': 0.9068888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.8703703703703703, 'test_acc': 0.8490555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.6975308641975309, 'test_acc': 0.7202222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.7962962962962963, 'test_acc': 0.8148333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.9012345679012346, 'test_acc': 0.8722777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.9012345679012346, 'test_acc': 0.8753888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9074074074074074, 'test_acc': 0.8531111111111112}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.8580246913580247, 'test_acc': 0.8445555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.8641975308641975, 'test_acc': 0.8683888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.9074074074074074, 'test_acc': 0.8755555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.7222222222222222, 'test_acc': 0.7442777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.8209876543209876, 'test_acc': 0.8135555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.8518518518518519, 'test_acc': 0.8768888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.9753086419753086, 'test_acc': 0.8890555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8703703703703703, 'test_acc': 0.8513888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8703703703703703, 'test_acc': 0.8796666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.8765432098765432, 'test_acc': 0.8648888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.8888888888888888, 'test_acc': 0.8681111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.845679012345679, 'test_acc': 0.8261111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.9259259259259259, 'test_acc': 0.8818333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.9012345679012346, 'test_acc': 0.8665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.8703703703703703, 'test_acc': 0.8596111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.9197530864197531, 'test_acc': 0.8518888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.8518518518518519, 'test_acc': 0.8543333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.9135802469135802, 'test_acc': 0.8942777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.845679012345679, 'test_acc': 0.8536111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.8827160493827161, 'test_acc': 0.8725}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.8765432098765432, 'test_acc': 0.8766666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.8703703703703703, 'test_acc': 0.8633333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9197530864197531, 'test_acc': 0.8906111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.8827160493827161, 'test_acc': 0.8529444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.9382716049382716, 'test_acc': 0.878}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.8703703703703703, 'test_acc': 0.8553888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9074074074074074, 'test_acc': 0.8991666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.8641975308641975, 'test_acc': 0.8503888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.8641975308641975, 'test_acc': 0.8504444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.8518518518518519, 'test_acc': 0.8318333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.8580246913580247, 'test_acc': 0.87}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.9012345679012346, 'test_acc': 0.8722222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9135802469135802, 'test_acc': 0.8576666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8271604938271605, 'test_acc': 0.8413333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.8827160493827161, 'test_acc': 0.8768888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.7283950617283951, 'test_acc': 0.7212222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9320987654320988, 'test_acc': 0.8883333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.8518518518518519, 'test_acc': 0.8620555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9074074074074074, 'test_acc': 0.8631111111111112}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9197530864197531, 'test_acc': 0.8875}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.8950617283950617, 'test_acc': 0.8548888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.9074074074074074, 'test_acc': 0.8728333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.8765432098765432, 'test_acc': 0.8635}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.8580246913580247, 'test_acc': 0.8711111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.8580246913580247, 'test_acc': 0.8634444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.8950617283950617, 'test_acc': 0.8767222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.8703703703703703, 'test_acc': 0.8747222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.7716049382716049, 'test_acc': 0.7654444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.9012345679012346, 'test_acc': 0.8563888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.8641975308641975, 'test_acc': 0.8346666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.8827160493827161, 'test_acc': 0.8635555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.8271604938271605, 'test_acc': 0.8395}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.9320987654320988, 'test_acc': 0.9001111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.8827160493827161, 'test_acc': 0.8447222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.8209876543209876, 'test_acc': 0.8115555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.8641975308641975, 'test_acc': 0.8787777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.9259259259259259, 'test_acc': 0.8745555555555555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8703703703703703, 'test_acc': 0.8671111111111112}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.845679012345679, 'test_acc': 0.8416111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8950617283950617, 'test_acc': 0.8804444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.8580246913580247, 'test_acc': 0.8275555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8703703703703703, 'test_acc': 0.8789444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.9259259259259259, 'test_acc': 0.8733333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.9074074074074074, 'test_acc': 0.8634444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8703703703703703, 'test_acc': 0.8527777777777777}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8950617283950617, 'test_acc': 0.8435}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.8827160493827161, 'test_acc': 0.8618333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9135802469135802, 'test_acc': 0.8898888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8580246913580247, 'test_acc': 0.8711111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8580246913580247, 'test_acc': 0.8578333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9259259259259259, 'test_acc': 0.8846666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8580246913580247, 'test_acc': 0.8726666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.7654320987654321, 'test_acc': 0.7868888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.9012345679012346, 'test_acc': 0.8576111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.8703703703703703, 'test_acc': 0.8382777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.8641975308641975, 'test_acc': 0.8594444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8518518518518519, 'test_acc': 0.8557222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.9012345679012346, 'test_acc': 0.8965}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.9074074074074074, 'test_acc': 0.8651666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.8333333333333334, 'test_acc': 0.8109444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.7962962962962963, 'test_acc': 0.8121111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.9320987654320988, 'test_acc': 0.8768888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8950617283950617, 'test_acc': 0.8680555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.845679012345679, 'test_acc': 0.8546666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9506172839506173, 'test_acc': 0.8881666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.8641975308641975, 'test_acc': 0.8409444444444445}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8395061728395061, 'test_acc': 0.8636666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.8703703703703703, 'test_acc': 0.8587777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9012345679012346, 'test_acc': 0.8672777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.8888888888888888, 'test_acc': 0.8468888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.9074074074074074, 'test_acc': 0.8268888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.8827160493827161, 'test_acc': 0.8452777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.8950617283950617, 'test_acc': 0.8629444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.8333333333333334, 'test_acc': 0.8001111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8518518518518519, 'test_acc': 0.8536111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.9382716049382716, 'test_acc': 0.8801111111111111}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.8209876543209876, 'test_acc': 0.8313888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8580246913580247, 'test_acc': 0.8512222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.8641975308641975, 'test_acc': 0.8512222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.9012345679012346, 'test_acc': 0.8616666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9197530864197531, 'test_acc': 0.8799444444444444}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.8827160493827161, 'test_acc': 0.8722222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.8827160493827161, 'test_acc': 0.8687222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.8703703703703703, 'test_acc': 0.8306666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.8395061728395061, 'test_acc': 0.849}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.8395061728395061, 'test_acc': 0.8505}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.8827160493827161, 'test_acc': 0.8532777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9074074074074074, 'test_acc': 0.8552222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.845679012345679, 'test_acc': 0.8521666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9382716049382716, 'test_acc': 0.8860555555555556}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8518518518518519, 'test_acc': 0.8228888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8703703703703703, 'test_acc': 0.8712222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.8580246913580247, 'test_acc': 0.8547222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.9012345679012346, 'test_acc': 0.8587777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.8765432098765432, 'test_acc': 0.8587777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.8950617283950617, 'test_acc': 0.8502777777777778}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.9012345679012346, 'test_acc': 0.8627222222222222}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8765432098765432, 'test_acc': 0.8531111111111112}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.8641975308641975, 'test_acc': 0.8517222222222223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.8641975308641975, 'test_acc': 0.8453888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.8827160493827161, 'test_acc': 0.8523333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.8024691358024691, 'test_acc': 0.8425}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.8765432098765432, 'test_acc': 0.8643333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.9197530864197531, 'test_acc': 0.8703888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.9074074074074074, 'test_acc': 0.8591666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.8518518518518519, 'test_acc': 0.8515}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8518518518518519, 'test_acc': 0.8483888888888889}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.8641975308641975, 'test_acc': 0.866}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.8580246913580247, 'test_acc': 0.8323888888888888}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.8827160493827161, 'test_acc': 0.8513333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.8271604938271605, 'test_acc': 0.8633333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.8950617283950617, 'test_acc': 0.8578333333333333}]
finetune3 = [{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8864197530864197, 'test_acc': 0.8854444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9049382716049382, 'test_acc': 0.907}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9407407407407408, 'test_acc': 0.9337777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8740740740740741, 'test_acc': 0.8771666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9067901234567901, 'test_acc': 0.8946111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.8820987654320988, 'test_acc': 0.8821111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.9098765432098765, 'test_acc': 0.8971111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.895679012345679, 'test_acc': 0.9038333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.8444444444444444, 'test_acc': 0.8475555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.8567901234567902, 'test_acc': 0.8586111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.6438271604938272, 'test_acc': 0.6415555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.8537037037037037, 'test_acc': 0.8623888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.9061728395061729, 'test_acc': 0.9058333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.6691358024691358, 'test_acc': 0.6687777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.8858024691358025, 'test_acc': 0.8843333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.8901234567901235, 'test_acc': 0.8796666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.8771604938271605, 'test_acc': 0.8622222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.8524691358024692, 'test_acc': 0.8521111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.8895061728395062, 'test_acc': 0.887}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.8876543209876543, 'test_acc': 0.8757222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8438271604938271, 'test_acc': 0.8368333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.8444444444444444, 'test_acc': 0.8416111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9450617283950618, 'test_acc': 0.9364444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.8790123456790123, 'test_acc': 0.8802222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9074074074074074, 'test_acc': 0.8934444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.8796296296296297, 'test_acc': 0.8800555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9092592592592592, 'test_acc': 0.8952222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.8851851851851852, 'test_acc': 0.8810555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.8537037037037037, 'test_acc': 0.8529444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.7314814814814815, 'test_acc': 0.7258333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9123456790123456, 'test_acc': 0.8966666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.8981481481481481, 'test_acc': 0.9033333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.9074074074074074, 'test_acc': 0.9096111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.7092592592592593, 'test_acc': 0.7063333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.7487654320987654, 'test_acc': 0.7626111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.8925925925925926, 'test_acc': 0.8862777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.8537037037037037, 'test_acc': 0.8462777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.8969135802469136, 'test_acc': 0.8955555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.8444444444444444, 'test_acc': 0.8355}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.8907407407407407, 'test_acc': 0.8797777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8524691358024692, 'test_acc': 0.8476666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8401234567901235, 'test_acc': 0.8355}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9234567901234568, 'test_acc': 0.9167222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.8969135802469136, 'test_acc': 0.8976666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8290123456790124, 'test_acc': 0.8234444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.8759259259259259, 'test_acc': 0.8838333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.7635802469135803, 'test_acc': 0.7512777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.8944444444444445, 'test_acc': 0.8968333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.8969135802469136, 'test_acc': 0.8916111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.854320987654321, 'test_acc': 0.8603333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.8666666666666667, 'test_acc': 0.8585555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9067901234567901, 'test_acc': 0.9032777777777777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9055555555555556, 'test_acc': 0.9117777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.8802469135802469, 'test_acc': 0.8751666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.7771604938271605, 'test_acc': 0.7813333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.7820987654320988, 'test_acc': 0.7922777777777777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.8604938271604938, 'test_acc': 0.8602222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.8925925925925926, 'test_acc': 0.8955}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.8561728395061728, 'test_acc': 0.8546111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9092592592592592, 'test_acc': 0.9017222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8641975308641975, 'test_acc': 0.8660555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9018518518518519, 'test_acc': 0.8971111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9067901234567901, 'test_acc': 0.9031666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.7080246913580247, 'test_acc': 0.6991111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.8993827160493827, 'test_acc': 0.8913888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.8574074074074074, 'test_acc': 0.8584444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.8734567901234568, 'test_acc': 0.8651111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.8975308641975308, 'test_acc': 0.8926111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.8808641975308642, 'test_acc': 0.8752777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.8617283950617284, 'test_acc': 0.8644444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.8808641975308642, 'test_acc': 0.8687777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.8611111111111112, 'test_acc': 0.8722222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.9074074074074074, 'test_acc': 0.9109444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.8765432098765432, 'test_acc': 0.8661111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.8592592592592593, 'test_acc': 0.8631111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.8006172839506173, 'test_acc': 0.8011666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.8654320987654321, 'test_acc': 0.8628888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.8234567901234567, 'test_acc': 0.8325}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.854320987654321, 'test_acc': 0.8551111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.8697530864197531, 'test_acc': 0.8612777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8555555555555555, 'test_acc': 0.8623333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8537037037037037, 'test_acc': 0.8528888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8765432098765432, 'test_acc': 0.8778333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.7111111111111111, 'test_acc': 0.7078333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8950617283950617, 'test_acc': 0.8900555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.8790123456790123, 'test_acc': 0.881}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.8672839506172839, 'test_acc': 0.863}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8481481481481481, 'test_acc': 0.8375}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8462962962962963, 'test_acc': 0.8423888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.8623456790123457, 'test_acc': 0.8609444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.8753086419753087, 'test_acc': 0.8603333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8654320987654321, 'test_acc': 0.8756666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8987654320987655, 'test_acc': 0.9006111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.8388888888888889, 'test_acc': 0.8267222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8827160493827161, 'test_acc': 0.8907222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.7839506172839507, 'test_acc': 0.7908333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.8469135802469135, 'test_acc': 0.8422777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.8160493827160494, 'test_acc': 0.8245}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.8716049382716049, 'test_acc': 0.8726666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8666666666666667, 'test_acc': 0.8550555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8660493827160494, 'test_acc': 0.8728888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.85, 'test_acc': 0.8530555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9265432098765433, 'test_acc': 0.9206666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.8351851851851851, 'test_acc': 0.8388333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8604938271604938, 'test_acc': 0.8601111111111112}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.854320987654321, 'test_acc': 0.8601666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.8746913580246913, 'test_acc': 0.8752777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.8530864197530864, 'test_acc': 0.8412222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.8561728395061728, 'test_acc': 0.8505555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.8617283950617284, 'test_acc': 0.8576111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.858641975308642, 'test_acc': 0.8452222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.7938271604938272, 'test_acc': 0.8045}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8388888888888889, 'test_acc': 0.8462222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.8845679012345679, 'test_acc': 0.8704444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.8524691358024692, 'test_acc': 0.8537222222222223}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8407407407407408, 'test_acc': 0.8432777777777778}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.8870370370370371, 'test_acc': 0.8807222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.8203703703703704, 'test_acc': 0.8264444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.8697530864197531, 'test_acc': 0.8650555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.8876543209876543, 'test_acc': 0.8777222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8703703703703703, 'test_acc': 0.8726111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8444444444444444, 'test_acc': 0.8525555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.891358024691358, 'test_acc': 0.8896666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8320987654320988, 'test_acc': 0.8323888888888888}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8493827160493828, 'test_acc': 0.8535555555555555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.8506172839506173, 'test_acc': 0.8548888888888889}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.8654320987654321, 'test_acc': 0.8599444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.8419753086419753, 'test_acc': 0.8439444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.8530864197530864, 'test_acc': 0.8516111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.8876543209876543, 'test_acc': 0.8754444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8666666666666667, 'test_acc': 0.8529444444444444}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.8524691358024692, 'test_acc': 0.8608333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.8623456790123457, 'test_acc': 0.8631666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.8907407407407407, 'test_acc': 0.8731111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.8432098765432099, 'test_acc': 0.8422222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.8425925925925926, 'test_acc': 0.8379444444444445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.8666666666666667, 'test_acc': 0.8500555555555556}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.8660493827160494, 'test_acc': 0.8787222222222222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.8382716049382716, 'test_acc': 0.8451111111111111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8469135802469135, 'test_acc': 0.8521111111111112}]
finetune4 = [{'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.941604938271605, 'test_acc': 0.9386666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.948395061728395, 'test_acc': 0.9459444444444445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9541975308641976, 'test_acc': 0.9552222222222222}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9285185185185185, 'test_acc': 0.9268888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9388888888888889, 'test_acc': 0.9398333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9409876543209876, 'test_acc': 0.9385555555555556}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.945925925925926, 'test_acc': 0.9435555555555556}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9550617283950618, 'test_acc': 0.9570555555555555}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9306172839506173, 'test_acc': 0.9290555555555555}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9403703703703704, 'test_acc': 0.9402222222222222}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8551851851851852, 'test_acc': 0.8557222222222223}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9408641975308641, 'test_acc': 0.9391666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.952716049382716, 'test_acc': 0.9553888888888888}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9412345679012346, 'test_acc': 0.9385555555555556}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9349382716049383, 'test_acc': 0.9349444444444445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.944320987654321, 'test_acc': 0.9412777777777778}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9408641975308641, 'test_acc': 0.939}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9546913580246914, 'test_acc': 0.9556666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.937037037037037, 'test_acc': 0.9358888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9354320987654321, 'test_acc': 0.9353888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9335802469135802, 'test_acc': 0.9333888888888889}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9390123456790124, 'test_acc': 0.9360555555555555}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.952716049382716, 'test_acc': 0.9536111111111111}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.927283950617284, 'test_acc': 0.9251111111111111}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9330864197530864, 'test_acc': 0.9328333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9340740740740741, 'test_acc': 0.9325555555555556}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9330864197530864, 'test_acc': 0.9311666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9495061728395062, 'test_acc': 0.9501111111111111}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.922716049382716, 'test_acc': 0.9212222222222223}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9320987654320988, 'test_acc': 0.9322222222222222}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9050617283950617, 'test_acc': 0.9065}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9166666666666666, 'test_acc': 0.9145}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9264197530864198, 'test_acc': 0.9285555555555556}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8914814814814814, 'test_acc': 0.8895555555555555}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8919753086419753, 'test_acc': 0.8941111111111111}]
finetune5 = [{'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9598048900963201, 'test_acc': 0.9593333333333334}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9597431464559151, 'test_acc': 0.9598333333333333}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9620276611509014, 'test_acc': 0.9622777777777778}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9560385280316127, 'test_acc': 0.9558333333333333}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9575203754013336, 'test_acc': 0.9577777777777777}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.961163250185231, 'test_acc': 0.9605555555555556}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9575821190417387, 'test_acc': 0.9575555555555556}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9591257100518646, 'test_acc': 0.9596111111111111}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9551741170659422, 'test_acc': 0.9550555555555555}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9567794517164732, 'test_acc': 0.9569444444444445}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9569646826376883, 'test_acc': 0.9570555555555555}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9606075574215855, 'test_acc': 0.9602777777777778}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.966102741417634, 'test_acc': 0.9659444444444445}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9565942207952581, 'test_acc': 0.9558888888888889}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9591257100518646, 'test_acc': 0.9587777777777777}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.963015559397382, 'test_acc': 0.9626111111111111}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9627685848357619, 'test_acc': 0.9625}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9595579155346999, 'test_acc': 0.9596666666666667}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9586935045690294, 'test_acc': 0.9581111111111111}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9591257100518646, 'test_acc': 0.9587777777777777}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9605458137811805, 'test_acc': 0.9600555555555556}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9604223265003705, 'test_acc': 0.9603888888888888}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9644356631266979, 'test_acc': 0.9643333333333334}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9527661150901457, 'test_acc': 0.9527777777777777}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9546184243022968, 'test_acc': 0.9547222222222222}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9540009878982465, 'test_acc': 0.9536666666666667}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9540627315386515, 'test_acc': 0.9532777777777778}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9601136082983452, 'test_acc': 0.9601666666666666}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.946900469251667, 'test_acc': 0.9467222222222222}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9543097061002717, 'test_acc': 0.954}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9304766609039269, 'test_acc': 0.931}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9389972832798221, 'test_acc': 0.9391111111111111}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9423931835020993, 'test_acc': 0.9421666666666667}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.919301061990615, 'test_acc': 0.9191666666666667}, {'lr': 0.001, 'sampled_percentage': 1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9208446530007409, 'test_acc': 0.9206666666666666}]

In [15]:
results = finetune1 + finetune2 + finetune3 + finetune4 + finetune5

In [17]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_reinitfalse.json', 'w') as f:
    json.dump(results, f)
results = results[1:]